Name: **Enis Simsar (2014400219)**

I hereby declare that I observed the honour code of the university when preparing the homework.

## Programming Homework

In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider letters 'a-z'. Capital letters 'A-Z' are mapped to the corresponding ones. All remaining letters, symbols, numbers, including spaces, are denoted by '.'.


We have a probability table $T$ where $T_{i,j} = p(x_t = j | x_{t-1} = i)$  transition model of letters in English text for $t=1,2 \dots N$. Assume that the initial letter in a string is always a space denoted as $x_0 = \text{'.'}$. Such a model where the probability table is always the same is sometimes called a stationary model.

1. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$
1. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 
1. Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
1. Discuss how you can improve the model to get better estimations.

In [2]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [3]:
import csv
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
print(T[letter2idx['q']][letter2idx['u']])
display(Latex(r"$p(x_t | x_{t-1} = \text{'a'})$"))
for c,p in zip(alphabet,T[letter2idx['a']]):
    print(c,p)

Example


<IPython.core.display.Latex object>

0.9949749


<IPython.core.display.Latex object>

a 0.0002835
b 0.0228302
c 0.0369041
d 0.0426290
e 0.0012216
f 0.0075739
g 0.0171385
h 0.0014659
i 0.0372661
j 0.0002353
k 0.0110124
l 0.0778259
m 0.0260757
n 0.2145354
o 0.0005459
p 0.0195213
q 0.0001749
r 0.1104770
s 0.0934290
t 0.1317960
u 0.0098029
v 0.0306574
w 0.0088799
x 0.0009562
y 0.0233701
z 0.0018701
. 0.0715219


### Solutions

In [4]:
import numpy as np

In [71]:
PRE_CALC_T_TABLE = 100

In [72]:
T = np.array(T).astype(np.float)
get_prob = lambda t, t_1: T[letter2idx[t_1 if t_1 in alphabet else '.']][letter2idx[t if t in alphabet else '.']]
pre_calc_T = np.zeros((PRE_CALC_T_TABLE, T.shape[0], T.shape[1]))
pre_calc_T[0] = T
for i in range(1, PRE_CALC_T_TABLE):
    pre_calc_T[i] = pre_calc_T[i - 1].dot(T)

**1.** For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$

In [73]:
def generate_string(str_len):
    generated_string, generated_string_prob = '', 1
    for i in range(str_len):
        prev_char = generated_string[i-1] if i > 0 else '.'
        rand_p = np.random.uniform(low=0, high=T[letter2idx[prev_char]].max())
        for char in alphabet:
            prob = get_prob(char, prev_char)
            if rand_p <= prob:
                generated_string_prob *= prob
                generated_string += char
                break
    return generated_string, generated_string_prob

generated_string, generated_string_prob = generate_string(3)
print("Generated str:\t {0} \nwith prob:\t {1}".format(generated_string, generated_string_prob))

Generated str:	 tha 
with prob:	 0.00773980806778933


**2.** Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc.

Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 

In [97]:
def fill_string(missing_str):
    missing_str = '.' + missing_str
    filled_str, filled_str_prob = '.', 1
    for i in range(len(missing_str) - 1):
        prev_char = filled_str[i]
        curr_char = missing_str[i+1]
        if curr_char not in alphabet:
            right_prob = T.copy()
            j = i + 2
            count = 0
            while j < len(missing_str) and missing_str[j] not in alphabet:
                count += 1
                j += 1
            right_prob = pre_calc_T[count]
            right_letter = missing_str[j] if j < len(missing_str) else '.'
            right_prob_with_elem = right_prob[:,letter2idx[right_letter]]
            left_prop_with_elem = T[letter2idx[prev_char]]
            
            chain_prob = pre_calc_T[count+1]
            
            result = right_prob_with_elem * left_prop_with_elem / chain_prob[letter2idx[prev_char]][letter2idx[right_letter]]
            
            rand_p = np.random.uniform(low=0, high=result.max())
            for index in range(len(result)):
                if rand_p <= result[index]:
                    filled_str_prob *= result[index]
                    filled_str += alphabet[index]
                    break
        else:
            filled_str += curr_char
    return filled_str[1:], filled_str_prob
# th__br__n.f_x.
for test_str in test_strings:
    filled_str, filled_str_prob = fill_string(test_str)
    print("Given str:\t {0} \nFilled str:\t {1} \nFilled str prob: {2}\n----------".format(test_str, filled_str, filled_str_prob))

Given str:	 th__br__n.f_x. 
Filled str:	 the.brean.fex. 
Filled str prob: 0.024830577322212846
----------
Given str:	 _u_st__n_.to_be._nsw_r__ 
Filled str:	 ouasthand.tombe.answared 
Filled str prob: 1.3143255942279403e-06
----------
Given str:	 i__at_._a_h_n_._e_r_i_g 
Filled str:	 ingata.cathand.bearaing 
Filled str prob: 5.167412697041578e-09
----------
Given str:	 q___t.___z._____t.__.___.__. 
Filled str:	 qur.t.thaz.the.at.ad.and.ad. 
Filled str prob: 5.858983418512638e-12
----------


**3.** Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.

**4.** Discuss how you can improve the model to get better estimations.